In [24]:
import requests
import numpy as np
from PIL import Image
import json
import base64
# Function to preprocess the image
def preprocess_image(image_path):
    # Load the image
    image = Image.open(image_path)
    # Resize the image to 224x224 (the input size expected by MobileNetV2)
    image = image.resize((224, 224))
    # Convert the image to a numpy array
    image_array = np.array(image)
    # Normalize the image array to [0, 1]
    image_array = image_array / 255.0
    # Add a batch dimension
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

# Function to make an inference request to OpenVINO
def predict(image_array, inference_url):
    # Convert the image array to a list of floats
    image_list = image_array.astype(np.float32).tolist()
    
    # Prepare the payload
    payload = {
        "inputs": [
            {
                "name": "input",
                "shape": [1, 224, 224, 3],
                "datatype": "FP32",
                "data": image_list
            }
        ]
    }
    
    # Prepare the headers
    headers = {'Content-Type': 'application/json'}
    
    # Make a POST request to the inference URL
    response = requests.post(inference_url, headers=headers, json=payload)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON
        predictions = response.json()
        return predictions
    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.text)
        return None
# Path to the input image
image_path = 'cat.jpg'
# URL of the OpenVINO inference endpoint
inference_url = 'https://test-nageshrathodiot-dev.apps.sandbox-m2.ll9k.p1.openshiftapps.com/v2/models/test/infer'


# Preprocess the image
image_array = preprocess_image(image_path)

# Make an inference request
output_prediction = predict(image_array, inference_url)

# Load the imagenet class index file
with open('imagenet_class_index.json') as f:
    class_idx = json.load(f)


# Extract the prediction scores from the output
predictions = output_prediction['outputs'][0]['data']

# Convert the list to a numpy array for easier manipulation
predictions_np = np.array(predictions)

# Find the index of the maximum prediction score
max_index = np.argmax(predictions_np)

# Get the human-readable label from the class index
label = class_idx[str(max_index)][1]

# Print the prediction
print(f'The predicted class is: {label}')

The predicted class is: Egyptian_cat
